# Introduction

# Creating the initial position file using watershed pourpoints
The following code read the pour points extracted from Arc gis from the intersection of watersheds with lake huron boundary.
 We are using PyLag Create initial position function to Create a release Zone for each of the Pour points. Before creating the initial positions, We want to add a condition to make a the relase zone that are in the boundary of Lake Huron.
 
 Then using Pylag create initial position file it will create a release zone for each pour point/groupid, in the end each group_id will contain sets of release zone 
 6/3/2024

#### Part 1-1: Import required libraries

In [2]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
from pylag.processing.coordinate import get_epsg_code, utm_from_lonlat, lonlat_from_utm
from pylag.processing.coordinate import utm_from_lonlat, lonlat_from_utm
from pylag.processing.release_zone import create_release_zones_along_cord
from pylag.processing.plot import create_figure, colourmap
from pylag.processing.plot import FVCOMPlotter
from pylag.processing.release_zone import create_release_zone
from pylag.processing.input import create_initial_positions_file_multi_group
from pylag.processing.input import create_initial_positions_file_single_group

# Input path/ files

In [27]:
# path to pourpoint 
pourpoints = r"/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/pourpoints"

init_file = os.path.join(pourpoints, 'WatershedPourpoints_NAD1983.dat')


# path to the release zone that intersect to Lake Huron
# path to release zone shapefile
release_zone = r"/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/particle_tracking_output"
intersection = "initial_positions_releasezone_intersection.shp"
release_zone_intersect = os.path.join(release_zone, 'initial_positions_releasezone_intersection.shp')


# Output path

In [3]:
out_path = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position'


# temporary file to store the release zone
temp_out = '/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/particle_tracking_output'
temp_out_name = os.path.join(temp_out, 'initial_positions_releasezone.shp')

# name of the output file that is multigrouped and ready to be used in FVCOM
output_file_initial_releasezone_multigroup = os.path.join(out_path, 'initial_positions_releasezone_intersection_multigroup.dat')
#out_name_multi= os.path.join(output_file_initial_releasezone_multigroup, 'initial_positions_multigroup.dat')


# Evironments
create directory/Folders

#### Part1-2: Read the initial position file(from watreshed and lake intersections) and rename the columns

In [ ]:
#TODO: make sure defining all the paths to the top 

In [ ]:
# define the path to the input file
data_dir ='/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position'
# the structure is 3 column with group_id, lon, lat, depth
init_file = os.path.join(data_dir, 'WatershedPourPoints_LH_copy_multigroup.dat')

# define the path to the output directory
# Create run directory
simulation_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron'
try:
    os.makedirs(simulation_dir)
except FileExistsError:
    pass



In [ ]:
# specify the outputs:


# Parameters/ temporary

In [ ]:
# specify the parameters to Create release zone
# Release zone  
radius = 0.01
# target number of particles
n_particles_target = 100

# Release depths
depth_below_surface = 0.0

# Reading the data

In [ ]:
# reading the pouropoints file
df_pourpoint = pd.read_csv(init_file, skiprows=0,sep=' ',header=None)
# rename the columns
df_pourpoint.rename(columns={0:"group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)

# Main Function 

# Create a release zone and save it

In [13]:
# list to accumulate release zones
release_zones = []

# Loop through the rows of the DataFrame
for index, row in df_pourpoint.iterrows():
    group_id = row['group_id']
    lat = row['lat']
    lon = row['lon']
    
    # Create the release zone
    surface_release_zone = create_release_zone(group_id = group_id,
                                           radius = radius,
                                           centre = [lon, lat],
                                           n_particles = n_particles_target,
                                           depth = depth_below_surface,
                                           random = False)
    
    # accumulate the release zones
    release_zones.append(surface_release_zone)
    


# ################ to the top
#  # Create input sub-directory / movwe this to output directory
# input_dir = os.path.join(simulation_dir, 'input/initial_position')
# os.makedirs(input_dir, exist_ok=True)

#     # Output filename
# file_name = os.path.join(input_dir, 'WatershedPourPoints_LH_multigroup_3.dat')

###################################################

# Write data to file
create_initial_positions_file_multi_group(output_file_initial_releasezone, release_zones)


In [ ]:
/home/abolmaal/data/FVCOME_OUTPUT/Nutrientflux_Project/Particle_Tracking/1_Create_Initial_Position_multigroup.ipynb

'/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position/initial_positions_releasezone.dat'

# Insure the points are placed inside Lake

In this part I save the initial position file as shape file and I used arcpy to remove the points that are not inside Lake Huron. 

Code for removing the points is in DirectNutrientLoad directory and it's called Initial_position_removeoutsidelakepoints

In [22]:
# read file name
# temp name
df_temp = pd.read_csv(output_file_initial_releasezone, skiprows=1,sep=' ',header=None)
# rename the columns
df_temp.rename(columns={0:"group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)
df_temp.head()

,group_id,lon,lat,depth
0,0.0,275.334153,45.735069,0.0
1,0.0,275.328153,45.737069,0.0
2,0.0,275.330153,45.737069,0.0
3,0.0,275.332153,45.737069,0.0
4,0.0,275.334153,45.737069,0.0


In [25]:
# save the dataframe as a shapefile in temp_out directory with temp_out_name
gdf = gpd.GeoDataFrame(df_temp, geometry=gpd.points_from_xy(df_temp.lon, df_temp.lat))
gdf.crs = 'EPSG:4326'
gdf.to_file(temp_out_name)

# 3-Solve an error related to PyLag

My Lat and lon number has more than 10 digits and when I run it through Pylag it gives me this error:ValueError: invalid literal for int() with base 10: '0.0' so to avoid this I want to read Lat and Lon and round them by 10


Once I removed the points outside Lake Boundary I read the cleaned point here again 
- make sure Longitudes are in 360 degree 
- Lat and Lon are rounded to 10 
- Save the file as dat file



In [41]:
# read the release zone shapefile that intersect with Lake Huron
release_zone_intersect = gpd.read_file(release_zone_intersect)
release_zone_intersect.head()

DriverError: group_id         lon        lat  depth4849       0.0  635.334153  45.755069    0.04825       0.0  635.328153  45.749069    0.04824       0.0  635.326153  45.749069    0.04823       0.0  635.342153  45.747069    0.04822       0.0  635.340153  45.747069    0.0...        ...         ...        ...    ...2292     147.0  636.532207  43.740642    0.02291     147.0  636.530207  43.740642    0.02290     147.0  636.528207  43.740642    0.02309     147.0  636.532207  43.742642    0.02244     147.0  636.532207  43.734642    0.0[4850 rows x 4 columns]: File name too long

In [ ]:
# read the release zone shapefile that intersect with Lake Huron
release_zone_intersect = gpd.read_file(release_zone_intersect)

# if lon column is less than 0 add 360 to it
release_zone_intersect['lon'] = release_zone_intersect['lon'] 

release_zone_intersect['lon'] = release_zone_intersect['lon'] + 360

# save the updated_gdf as a dat file and keep only the group_id, lon, lat, and depth columns
release_zone_intersect = release_zone_intersect[['group_id', 'lon', 'lat', 'depth']]
# sort the data from group_id
release_zone_intersect = release_zone_intersect.sort_values(by=['group_id'])
# if df[lon] and df[lat] are greater than 10 digits round them to 10 digits
release_zone_intersect['lon'] = release_zone_intersect['lon'].round(10)
release_zone_intersect['lat'] = release_zone_intersect['lat'].round(10)
release_zone_intersect.head()

,group_id,lon,lat,depth
4849,0.0,635.334153,45.755069,0.0
4825,0.0,635.328153,45.749069,0.0
4824,0.0,635.326153,45.749069,0.0
4823,0.0,635.342153,45.747069,0.0
4822,0.0,635.340153,45.747069,0.0


# Save outputs

In [39]:
release_zone_intersect.to_csv(output_file_initial_releasezone_multigroup, sep=' ', index=False, header=False)

In [7]:
# save the updated_gdf as a dat file and skip the header
updated_gdf.to_csv('/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/input/initial_position/WatershedPourPoints_LH_multigroup_rounded_initialposition_intersection2.dat', sep=' ', index=False, header=False)